In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import dataset
import predict

In [2]:
data = pd.read_csv('data/EPL_processed_results.csv', index_col=0)
X_train, y_train, X_val, y_val, X_test, y_test = dataset.generate_train_val_test_sets(data)

Load a trained model

In [3]:
predictor = predict.Predictor()
predictor.load_saved_model('model')

2022-03-06 16:42:25.041817: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Evaluate on test set

In [5]:
loss, acc = predictor.evaluate(X_test, y_test)

8/8 [==============================] - 1s 4ms/step - loss: 0.9863 - accuracy: 0.5294


Predict on a new sample